In [9]:
import pandas as pd
import numpy as np
from Functions.LoadData import get_data
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
from tqdm.notebook import tqdm
import webbrowser
import os
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [10]:
df_combined, df_fine_patient_phases, medication_settings, medTimes = get_data('EM2')

C:\Users\merlin.koehler\Desktop\EMERGENT\Functions\LoadData.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fine_patient_phases['end_date'].iloc[:-1]=df_fine_patient_phases['start_date'].iloc[1:]
C:\Users\merlin.koehler\Desktop\EMERGENT\Functions\LoadData.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fine_patient_phases['end_date'].iloc[-1]=df_combined['time'].iloc[-1]


In [11]:
phases = df_fine_patient_phases['phase'].unique()
for phase in phases:
    start_date = df_fine_patient_phases[df_fine_patient_phases['phase'] == phase]['start_date'].values[0]
    end_date = df_fine_patient_phases[df_fine_patient_phases['phase'] == phase]['end_date'].values[0]
    medication = medication_settings[phase]

    for date in pd.date_range(start_date, end_date):
        for med in medication.columns[2:]:
            for time in medication['time']:
                #combine date and time
                datetime = pd.to_datetime(str(date.date()) + ' ' + str(time))

                # get the medication value
                med_value = medication[medication['time'] == time][med].values[0]

                # check if the medication already in df_combined as a column
                if med not in df_combined.columns:
                    df_combined[med] = np.nan

                # add the medication value to the df_combined
                df_combined.loc[df_combined['time'] == datetime, med] = med_value
        
        df_combined.loc[df_combined[med] <= 0, med] = np.nan

In [12]:
fine_phases = df_fine_patient_phases['phase'].unique()
dbs_change_dates = []


for i in range(1,len(fine_phases)):
    current_phase = df_fine_patient_phases[df_fine_patient_phases['phase'] == fine_phases[i]].iloc[:,1:-2].to_numpy()
    previous_phase = df_fine_patient_phases[df_fine_patient_phases['phase'] == fine_phases[i-1]].iloc[:,1:-2].to_numpy()
    if(np.any(current_phase - previous_phase) != 0):
        dbs_change_dates.append(df_fine_patient_phases['start_date'][df_fine_patient_phases['phase'] == fine_phases[i]].iloc[0])

In [13]:
df_dbs_settings = df_fine_patient_phases[df_fine_patient_phases['start_date'].isin(dbs_change_dates)].iloc[:,1:-2]

In [14]:
medication_names = df_combined.columns[11:].to_list()
medication_times = {}
for medication in medication_names:
    if(len(df_combined[df_combined[medication] > 0]) > 0):
        medication_times[medication] = df_combined[df_combined[medication] > 0][['time', medication]].reset_index(drop=True)
    else:
        medication_names.remove(medication)

In [15]:
def plotly_line(df_combined, dbs_change_dates, df_dbs_settings, medication_names, medication_settings, column1, column2, agregation='10min', width=1200, height=1200, result_file='line.html'):
    df_column = pd.DataFrame()
    df_column[column1] = df_combined[['time', column1]].groupby(pd.Grouper(freq=agregation, key='time')).mean()
    df_column[column2] = df_combined[['time', column2]].groupby(pd.Grouper(freq=agregation, key='time')).mean()

    # px.line(df_column, x=df_column.index, y=[column1, column2], title=column1 + ' and ' + column2, width=width, height=height)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_column.index, y=df_column[column1], mode='lines', name=column1))
    fig.add_trace(go.Scatter(x=df_column.index, y=df_column[column2], mode='lines', name=column2))

    for i in range(len(dbs_change_dates)):
        date = dbs_change_dates[i]
        fig.add_shape(type="line",
                    x0=date, y0=0, x1=date, y1=max(df_column[column1].max(), df_column[column2].max()),
                    line=dict(color="red",width=3,dash="dashdot"))
        str_annotation = ''
        for column_name, column_value in df_dbs_settings.iloc[i,:].items():
            str_annotation += column_name + ': ' + str(column_value) + '<br>'
        fig.add_annotation(x=date,
                        y=max(df_column[column1].max(), df_column[column2].max()),
                        text="DBS change<br>" + str_annotation,
                        showarrow=True,
                        arrowhead=1,
                        yshift=10)


    for medication in medication_names:
        df_medication_date_time = medication_times[medication]
        # Triangle marker
        fig.add_trace(go.Scatter(x=df_medication_date_time['time'], y=[0]*len(df_medication_date_time), mode='markers', name=medication, marker=dict(color='green', size=10, symbol='triangle-up'), hovertext=df_medication_date_time[medication]))
    
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label="1m",
                        step="month",
                        stepmode="backward"),
                    dict(count=7,
                        label="1w",
                        step="day",
                        stepmode="backward"),
                    dict(count=1,
                        label="1d",
                        step="day",
                        stepmode="backward")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    fig.update_layout(
        title_text=column1 + ' and ' + column2)
    fig.write_html(result_file)

def plotly_heatmap(df_combined, dbs_change_dates, df_dbs_settings, column, agregation='15min', width=1200, height=1200, result_file='heatmap.html'):
    df_column = df_combined[['time', column]].groupby(pd.Grouper(freq=agregation, key='time')).mean()

    # split date and time: date as row, time as column
    df_aggregated = pd.DataFrame(df_column.index.to_series().dt.date.unique(), columns=['date'])
    df_aggregated = df_aggregated.set_index('date')

    for time in df_column.index.to_series().dt.time.unique():
        df_aggregated[time] = df_column[df_column.index.to_series().dt.time == time][column].to_numpy()
    
    # transpose dataframe
    df_aggregated = df_aggregated.transpose()
    
    # cmap=Reds
    fig = go.Figure()
    #fig = px.imshow(df_aggregated, title=column, labels=dict(x="Time", y="Date", color=column), width=width, height=height, color_continuous_scale='Reds')
    fig.add_trace(go.Heatmap(z=df_aggregated.to_numpy(), x=df_aggregated.columns, y=df_aggregated.index, colorscale='Reds'))

    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label="1m",
                        step="month",
                        stepmode="backward"),
                    dict(count=7,
                        label="1w",
                        step="day",
                        stepmode="backward"),
                    dict(count=1,
                        label="1d",
                        step="day",
                        stepmode="backward")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    
    for i in range(len(dbs_change_dates)):
        date = dbs_change_dates[i]
        #fig.add_vline(x=date, line_width=3, line_dash="dash", line_color="red")
        fig.add_shape(type="line",
                    x0=date, y0=df_aggregated.index.min(), x1=date, y1=df_aggregated.index.max(),
                    line=dict(color="red",width=3,dash="dashdot"))
        
        # for each column in df_dbs_settings.iloc[i,:]: do column_name: column_value new line
        str_annotation = ''
        for column_name, column_value in df_dbs_settings.iloc[i,:].items():
            str_annotation += column_name + ': ' + str(column_value) + '<br>'

        fig.add_annotation(x=date,
                        y=df_aggregated.index.max(),
                        text="DBS change:<br>" + str_annotation,
                        showarrow=True,
                        arrowhead=1,
                        yshift=10)

    for medication in medication_names:
        df_medication_date_time = medication_times[medication]
        for date in df_medication_date_time['time'].dt.date.unique():
            for time in df_medication_date_time[df_medication_date_time['time'].dt.date == date]['time'].dt.time.unique():
                # no legend
                fig.add_trace(go.Scatter(x=[date], y=[time], mode='markers', marker=dict(color='green', size=5, symbol='triangle-right'), showlegend=False, hovertext=df_medication_date_time[(df_medication_date_time['time'].dt.date == date) & (df_medication_date_time['time'].dt.time == time)][medication]))


    # add a title
    fig.update_layout(
        title_text=column)
    fig.write_html(result_file)

def plot_data(df, column1, column2, medication_names, medication_settings, agregation='15min', width='1900', height='1000'):
    width=int(width)
    height=int(height)
    plotly_line(df, dbs_change_dates, df_dbs_settings, medication_names, medication_settings, column1, column2, agregation, width, height,'line.html')
    plotly_heatmap(df, dbs_change_dates, df_dbs_settings, column1, agregation, width, height,'heatmap1.html')
    plotly_heatmap(df, dbs_change_dates, df_dbs_settings, column2, agregation, width, height,'heatmap2.html')

    url = 'file://' + os.path.realpath("line.html")
    webbrowser.open(url)

    url = 'file://' + os.path.realpath("heatmap1.html")
    webbrowser.open(url)

    url = 'file://' + os.path.realpath("heatmap2.html")
    webbrowser.open(url)

In [16]:
interact_manual(plot_data, df=fixed(df_combined), column1=df_combined.columns[1:-2], column2=df_combined.columns[1:-2], medication_names=fixed(medication_names), medication_settings=fixed(medication_settings), agregation=['1min', '5min', '10min', '15min', '30min', '1h', '2h', '3h', '6h', '12h', '1d'], width='1200', height='1200')

interactive(children=(Dropdown(description='column1', options=('probability_dyskinesia', 'probability_tremor',…

<function __main__.plot_data(df, column1, column2, medication_names, medication_settings, agregation='15min', width='1900', height='1000')>